# Lesson 2 : LangGraph Components

In [4]:
import os
os.system("clear")
abs_path = os.path.dirname(os.path.abspath(__name__))
os.chdir(abs_path)
print(f"Working directory: <{os.getcwd()}>")

Working directory: </home/tjamil/Desktop/Agents/Deeplearning_Ai/Ai Agents using Langgraph>


In [5]:
#from dotenv import load_dotenv
#_ = load_dotenv()

In [6]:
from setup_environment_r1 import set_environment_variables
set_environment_variables("LangGraph Basics - deeplearning.ai")

reading environment variables from: </home/tjamil/Desktop/Agents/Deeplearning_Ai/Ai Agents using Langgraph>
API Keys loaded and tracing set with project name:  LangGraph Basics - deeplearning.ai


In [12]:
from langgraph.graph import StateGraph, END

from typing import TypedDict, Annotated

import operator

from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
#from langchain_openai import ChatOpenAI
from utils import GroqChatLLM

from langchain_community.tools.tavily_search import TavilySearchResults

In [10]:
tool = TavilySearchResults(max_results=4) #increased number of results
print(type(tool))
print(tool.name)   # this is the name of the tool taht will be used by LLM

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


> If you are not familiar with python typing annotation, you can refer to the [python documents](https://docs.python.org/3/library/typing.html).

In [11]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

> Note: in `take_action` below, some logic was added to cover the case that the LLM returned a non-existent tool name. Even with function calling, LLMs can still occasionally hallucinate. Note that all that is done is instructing the LLM to try again! An advantage of an agentic organization.

In [13]:
class Agent:

    def __init__(self, model, tools, system=""):
        #setting up the system message
        self.system = system
        
        #defining the graph
        graph = StateGraph(AgentState)
        
        #adding nodes (llm, action/tools)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        
        #adding conditional edges (decide if action should be taken or not, and which node to go to)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        
        #adding edges
        graph.add_edge("action", "llm")
        
        #setting entry point
        graph.set_entry_point("llm")
        
        #compiling the graph (creates a runnable graph)
        self.graph = graph.compile()
        
        #defining the tools
        self.tools = {t.name: t for t in tools}
        
        #defining the model with tools binding
        self.model = model.bind_tools(tools)

    #defining the function for conditional edge 
    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    #defining 'llm node' function
    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    #defining 'action node' function
    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls    
        # .too_call is attribute returned aalongside message by functiona calling llms
        
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
                print(f"Result: {result}")

            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        
        print("Back to the model!")
        return {'messages': results}

In [14]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

#model = ChatOpenAI(model="gpt-3.5-turbo")  #reduce inference cost
model = GroqChatLLM()
abot = Agent(model, [tool], system=prompt)

In [1]:
from IPython.display import Image

#Image(abot.graph.get_graph().draw_png())

In [17]:
messages = [HumanMessage(content="What is the weather in sf?. Give just suffiucient information.")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_mw4e', 'type': 'tool_call'}
Result: [{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1726467744, 'localtime': '2024-09-15 23:22'}, 'current': {'last_updated_epoch': 1726467300, 'last_updated': '2024-09-15 23:15', 'temp_c': 13.9, 'temp_f': 57.0, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 18.8, 'wind_kph': 30.2, 'wind_degree': 270, 'wind_dir': 'W', 'pressure_mb': 1010.0, 'pressure_in': 29.82, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 83, 'cloud': 75, 'feelslike_c': 11.7, 'feelslike_f': 53.0, 'windchill_c': 10.6, 'windchill_f': 51.1, 'heatindex_c': 12.7, 'heatindex_f': 54.8, 'dewpoint_c': 12.3, 'dewpo

In [18]:
result

{'messages': [HumanMessage(content='What is the weather in sf?. Give just suffiucient information.'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_mw4e', 'function': {'arguments': '{"query": "weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 301, 'total_tokens': 334, 'completion_time': 0.102024874, 'prompt_time': 0.031200228, 'queue_time': 0.0018164409999999985, 'total_time': 0.133225102}, 'model_name': 'llama3-groq-70b-8192-tool-use-preview', 'system_fingerprint': 'fp_ee4b521143', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-cb2809d3-e88c-4e9e-af3d-fbcd77e8a354-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_mw4e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 301, 'output_tokens': 33, 'total_tokens': 334}),
  ToolMessage(content='[{\'url\': \'https://www.w

In [19]:
result['messages'][-1].content

'The current weather in San Francisco is partly cloudy with a temperature of 13.9°C (57.0°F). The wind is blowing from the west at 18.8 mph (30.2 kph). The humidity is at 83%, and the pressure is 1010.0 hPa.'

In [20]:
messages = [HumanMessage(content="What is the weather in SF and LA?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in SF'}, 'id': 'call_5mdg', 'type': 'tool_call'}
Result: [{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1726467744, 'localtime': '2024-09-15 23:22'}, 'current': {'last_updated_epoch': 1726467300, 'last_updated': '2024-09-15 23:15', 'temp_c': 13.9, 'temp_f': 57.0, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 18.8, 'wind_kph': 30.2, 'wind_degree': 270, 'wind_dir': 'W', 'pressure_mb': 1010.0, 'pressure_in': 29.82, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 83, 'cloud': 75, 'feelslike_c': 11.7, 'feelslike_f': 53.0, 'windchill_c': 10.6, 'windchill_f': 51.1, 'heatindex_c': 12.7, 'heatindex_f': 54.8, 'dewpoint_c': 12.3, 'dewpoint_f': 54.

In [21]:
result['messages'][-1].content

'The current weather in San Francisco is partly cloudy with a temperature of 13.9°C (57.0°F). The wind is blowing from the west at 18.8 mph (30.2 kph). In Los Angeles, the weather is overcast with a temperature of 19.4°C (66.9°F). The wind is coming from the south at 4.7 mph (7.6 kph).'

In [23]:
# Note, the query was modified to produce more consistent results. 
# Results may vary per run and over time as search information and models change.

query = "Who won the super bowl in 2024? In what state is the winning team headquarters located? \
What is the GDP of that state? Answer each question." 
messages = [HumanMessage(content=query)]

#model = ChatOpenAI(model="gpt-4o")  # requires more advanced model
model = GroqChatLLM()
abot = Agent(model, [tool], system=prompt)
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'who won the super bowl in 2024'}, 'id': 'call_pg72', 'type': 'tool_call'}
Result: [{'url': 'https://www.espn.com/nfl/story/_/id/39480722/49ers-chiefs-live-super-bowl-lviii-updates-moments-highlights', 'content': "With a relentless defense and opportune plays by their star quarterback -- including a pair of gutsy overtime scrambles -- the Chiefs won their third Super Bowl in five years in a 25-22 overtime victory against the San Francisco 49ers in only the second overtime game in Super Bowl history.\n Staff\nTaylor Swift supports Travis Kelce, chugs drink at Super Bowl LVIII10hTory Barron\nAfter posting a losing record in December, the Chiefs embraced an underdog, villain mentality throughout the postseason, upsetting three teams en route to their second consecutive Super Bowl title, becoming the ninth team to repeat as Super Bowl champions and first since the 2004 New England Patriots.\n ESPN\nSuper Bowl 2024 - Highligh

In [24]:
print(result['messages'][-1].content)

The Kansas City Chiefs won the Super Bowl in 2024, defeating the San Francisco 49ers in overtime with a score of 25-22. The Chiefs' headquarters are located in Kansas City, Missouri. The GDP of Missouri is approximately $301 billion as of 2020.
